In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
# from agents.dqn.agent import Agent, Config
# from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Basic constants and setting up environment

In [2]:
scenario = "convergence"

simTime = 10 # seconds
stepTime = 0.01  # seconds
history_length = 10


EPISODE_COUNT = 30
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": 10,
    "agentType": Agent.TYPE,
    "nonZeroStart": True,
    "scenario": "convergence",
    "nWifi": 7
}
print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

Steps per episode: 1000


In [3]:
# config = Config(buffer_size=2e3, batch_size=64, gamma=0.99, tau=1e-3, lr=5e-4)
config = Config(buffer_size=1.5e4*threads_no, batch_size=1024, gamma=0.99, tau=1e-3, lr_actor=6e-5, lr_critic=1e-3)

In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Observation space shape: (1, 'Box(10,)')
Action space shape: (1, 'Box(1,)')


### Creating and training agent

In [5]:
# import tensorflow as tf

# class Network(QNetworkTf):
#     def _inference(self):
#         with tf.variable_scope("inference_"+self.name):
#             layer = tf.layers.dense(self.input, 128, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 128, activation=tf.nn.relu)
# #             layer = tf.layers.batch_normalization(layer)
#             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 32, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, self.action_size)
#         return layer

In [6]:
agent = Agent(history_length, action_size=1, config=config)
# agent = Agent(Network, history_length, action_size=3, config=config)
# agent.set_epsilon(0.9, 0.01, 25)
teacher = Teacher(agent, env, 1)
# teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: window Mb sent", "Rew: normalized speed", "DQN", "Convergence", "Instances: 2", "Net: 128, 64, 32")
teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: collisions mb", "Rew: normalized speed", "DDPG", "Convergence", "Actor: 256, 128, 64", "Critic: 512, 256, 128", f"Instances: {threads_no}")

SCRIPT_RUNNING = False

COMET INFO: old comet version (1.0.45) detected. current: 1.0.50 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/af4a7250648c43a8a07a4d66a4c5af5e

100%|███████████████████████████████████████████████████████████| 1000/1000 [01:29<00:00, 11.18it/s, mb_sent=303.71 Mb]


Sent 303.71 Mb/s.	Mean speed: 30.37 Mb/s	Episode 1/30 finished



  3%|█▊                                                             | 28/1000 [00:02<01:14, 13.03it/s, mb_sent=9.59 Mb]

------- STARTED TRAINING -------


100%|███████████████████████████████████████████████████████████| 1000/1000 [01:44<00:00,  9.57it/s, mb_sent=354.79 Mb]


...

Sent 373.11 Mb/s.	Mean speed: 37.31 Mb/s	Episode 29/30 finished



100%|███████████████████████████████████████████████████████████| 1000/1000 [01:48<00:00,  9.23it/s, mb_sent=373.19 Mb]


Sent 373.19 Mb/s.	Mean speed: 37.32 Mb/s	Episode 30/30 finished



COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:                     Chosen CW: 103.0
COMET INFO:             Cumulative reward: -2.430419921875
COMET INFO:                Megabytes sent: 373.1930900000003
COMET INFO:                 Per-ep reward: -2.430419921875
COMET INFO:          Round megabytes sent: 0.39093
COMET INFO:                  Round reward: 0.021240234375
COMET INFO:                         Speed: 37.31930900000003
COMET INFO:                    actor_loss: -0.039362077
COMET INFO:                   critic_loss: 0.72469485
COMET INFO:                          loss: -0.03387805446982384
COMET INFO:         sys.gpu.0.free_memory: 7815766016
COMET INFO:     sys.gpu.0.gpu_utilization: 1
COMET INFO:        sys.gpu.0.total_memory: 11811160064
COMET INFO:         sys.gpu.0.used_memory: 3995394048
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take 

Training finished.
